In [36]:
!pip install --quiet transformers==4.8.1
!pip install --quiet sentencepiece==0.1.95
!pip install --quiet textwrap3==0.9.2
!pip install --quiet strsim==0.0.3
!pip install --quiet sense2vec==2.0.0
!pip install --quiet ipython-autotime
!pip install --quiet sentence-transformers==2.2.2
%load_ext autotime

from textwrap3 import wrap
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer
summary_model = T5ForConditionalGeneration.from_pretrained('t5-base')
summary_tokenizer = T5Tokenizer.from_pretrained('t5-base')

model_max_length=512 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summary_model = summary_model.to(device)

import random
import numpy as np

#not sure what this part does
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

import nltk
nltk.download('punkt')
nltk.download('brown')
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.tokenize import sent_tokenize

question_model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
question_tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_squad_v1')
question_model = question_model.to(device)

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  Retrieve sequence ids from a token list that has no special tokens added. This method is called when adding
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /roo

time: 35.7 s (started: 2023-03-09 16:42:36 +00:00)


In [ ]:
def get_question(context,answer,model,tokenizer):
    text = "context: {} answer: {}".format(context,answer)
    encoding = tokenizer.encode_plus(text,max_length=384, pad_to_max_length=False,truncation=True, return_tensors="pt").to(device)
    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

    outs = model.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=5,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  max_length=72)


    dec = [tokenizer.decode(ids,skip_special_tokens=True) for ids in outs]


    Question = dec[0].replace("question:","")
    Question= Question.strip()
    return Question

for wrp in wrap(text, 150):
    print (wrp)
print ("\n")

for answer in keywords:
    ques = get_question(text,answer,question_model,question_tokenizer)
    print (ques)
    print (answer.capitalize())
    print ("\n")

In [44]:
text = """In the year 1878 I took my degree of
Doctor of Medicine of the University of
London, and proceeded to Netley to go
through the course prescribed for surgeons in the army. Having completed my studies
there, I was duly attached to the Fifth Northumberland Fusiliers as Assistant Surgeon. The regiment
was stationed in India at the time, and before I
could join it, the second Afghan war had broken
out. On landing at Bombay, I learned that my corps
had advanced through the passes, and was already
deep in the enemy’s country. I followed, however,
with many other officers who were in the same
situation as myself, and succeeded in reaching Candahar in safety, where I found my regiment, and at
once entered upon my new duties." """

def generate(text, keywords):
  QA = []
  for answer in keywords:
    ques = get_question(text,answer,question_model,question_tokenizer)
    QA.append(ques)
  return QA
  
keywords = ['surgeons', 'Bombay', 'India', 'the regiment']

Questions = generate(text, keywords)

for i in range(0, len(keywords)):
  print(f"{keywords[i]} - {Questions[i]}")

surgeons - What was the course prescribed for in the army?
Bombay - Where did I learn that my corps had advanced through the passes?
India - Where was the Fifth Northumberland Fusiliers stationed?
the regiment - Who was stationed in India at the time?
time: 23.5 s (started: 2023-03-09 16:54:32 +00:00)
